# Análisis clúster y análisis de anomalías

## 1. Introducción
Una de las tareas del análisis de datos consiste en la identificación de agrupaciones naturales de individuos.

Para esto, normalmente se cuenta con una base de datos cuyas fílas son los individuos de interés
(personas, empresas, transacciones, etc.) y las columnas son sus características. Esto último, que existan muchas variables por cada registro, es muy relevante en esta parte del curso y justifíca el uso de metodologías

multivariadas de agrupación. Estas técnicas usualmente se grupan bajo el título de análisis clúster.
Para el aprendizaje de máquina (machine learning) el análisis clúster hace parte del aprendizaje no
supervisado porque no se tienen concepciones a priori sobre cómo conformar los grupos.

## 2. Análisis clúster

El objetivo del análisis clúster es la identificación de grupos de individuos (o de variables) en conjuntos de datos multidimensionales. Los individuos (variables) que pertenecen a un mismo grupo deben ser parecidos, mientras que los grupos deben ser diferentes entre sí. Aquí, el concepto de parecido (o diferente) dependende de algún tipo de distancia entre las observaciones.
El análisis clúster es popular siempre que exista interes en grupos de individuos (variables). Por ejemplo, en mercadeo, es importante hacer segmentaciones de clientes de acuerdo con sus características para después aplicar estrategias de ventas diferentes enfocadas a cada subgrupo.
En la práctica, se deben tener en cuenta las siguientes recomendaciones:
* En los datos, los elementos que se quieren agrupar deben estar organizados por fílas, mientras que las caracterésticas que definen a estos elementos van por columnas.
* Se debe evitar la presencia de valores perdidos. Si existen se deben imputar o eliminar la observación, teniendo muy en cuenta las posibles consecuencias en términos de sesgo en los datos.
* Las variables deben tener escalas similares.

Con respecto al último punto, los algoritmos de clustering, en general, son sensibles a la escala de las variables. Por ejemplo, en una base de datos de países, se pueden tener como variables el producto interno bruto (PIB) y el crecimiento. Aquí, el PIB suele venir en unidades monetarias, y tener valores incluso de billones, mientras que el crecimiento es un porcentaje. De esta forma, la escala de las variables es muy diferente y el algoritmo tenderá a agrupar con base en la variable de mayor escala, en este caso el PIB. Para evitar esto, las variables pueden estandarizarse (llevar a series con media 0 y varianza 1) o reescalarse (todas tienen el mismo mínimo y máximo).

> **📝 Nota:** Los procesos de estandarización o reescalamiento no aplican si las variables que describen al individuo son categóricas. En este caso, se debe recurrir a opciones distintas como la construcción de variables dummys (dicotómicas, de ceros y unos) o utilizar medidas de distancia capaces de tratar con este tipo de información.

## 2.1. Medidas de distancia

La distancia entre dos observaciones mide su cercanía (como era de esperar). Para un grupo de individuos, se puede obtener una matriz que contenga la distancia entre todos ellos. Como resultado, la matriz de distancias es cuadrada (igual número de filas y columnas) y tiene ceros en su diagonal principal.

La distancia entre un par de individuos 1 y 2, definidos a partir de p variables $x_{1i}, x_{2i}, ... x_{pi}$ con $i = 1,2$ se puede calcular como:

* Distancia Euclídea: $d (x_1, x_2) = \sqrt{\sum_{j=1}^{p} (x_{j1}-x_{j2})^2}$.
* Distancia Manhattan (o del taxista):  $d (x_1, x_2) = \sum_{j=1}^{p} |x_{j1}-x_{j2}|$.
* Distancia de Mahalanobis: $d (x_1, x_2) = \sqrt{{(x_{j1}-x_{j2})^T} * {\Sigma}^{-1} (x_{1}-x_{2})}$ donde $\Sigma$ es la matriz de varianzas y covarianzas entre $x_1$ y $x_2$.

También existen otras medidas basadas en la correlacion entre individuos que son utilizadas en contextos
específicos. En este caso, se considera que dos individuos son parecidos si tienen una alta correlación en sus características, aún cuando los valores que éstas tomen sean muy diferentes:
* Distancia de correlación Pearson: $d (x_1, x_2) = 1 - \dfrac{\sum_{j=1}^{p} (x_{j1} - \bar x)(x_{j2} - \bar x)}{\sqrt{\sum_{j=1}^{p} (x_{j1} - \bar x)^2(x_{j2} - \bar x)^2}}$.

Muchas de las medidas de distancia aplican únicamente a variables numéricas, pero en la práctica es
usual que las características de los individuos estén definidas como series ordinales o incluso categoricas.

Para estos casos se ha desarrollado la distancia de Gower, que agrupa las variables de un conjunto de datos de acuerdo con su naturaleza y aplica las medidas de distancias pertinentes según cada caso: para variables numéricas y ordinales se utiliza la distancia Manhattan (con un ajuste por empates para el segundo caso), y para variables nominales con k categorías se generan k variables indicadoras (dicotómicas) y luego se aplican medidas de similaridad adecuadas. Las distancias de cada caso se combinan de manera lineal, usualmente como un promedio simple.

## 3. Implementando algoritmos en Python

### 3.1. Carga y exploración de datos
A continuación se utilizarán datos empresariales con informacion de estados financieros. En primera instancia, se cargan y exploran los datos, para esto se utilizará la libreria de Pandas:


In [1]:
# Clonar repo de Git
!git clone https://github.com/dfbeltran/cursotomadecisiones.git

Cloning into 'cursotomadecisiones'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 47 (delta 7), reused 42 (delta 5), pack-reused 0
Unpacking objects: 100% (47/47), done.


In [2]:
# Importar librerias que se van a usar
import pandas as pd
from functools import reduce

# Cargar los datos
caratula = pd.read_csv('/content/cursotomadecisiones/datos/tema_4/Supersociedades SIREM 201706 - Caratula.csv',
                       encoding='latin-1', # Tipo de codificación del archivo (latin-1 dado a que tiene comas)
                       sep = ';') # Seleccionar separador 

efe = pd.read_csv('/content/cursotomadecisiones/datos/tema_4/Supersociedades SIREM 201706 - EFE.csv',
                       encoding='latin-1',
                       sep = ';')

eri = pd.read_csv('/content/cursotomadecisiones/datos/tema_4/Supersociedades SIREM 201706 - ERI.csv',
                       encoding='latin-1',
                       sep = ';')

esf = pd.read_csv('/content/cursotomadecisiones/datos/tema_4/Supersociedades SIREM 201706 - ESF.csv',
                       encoding='latin-1',
                       sep = ';')

ori = pd.read_csv('/content/cursotomadecisiones/datos/tema_4/Supersociedades SIREM 201706 - ORI.csv',
                       encoding='latin-1',
                       sep = ';')

In [3]:
caratula.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15955 entries, 0 to 15954
Data columns (total 12 columns):
 #   Column                                                         Non-Null Count  Dtype 
---  ------                                                         --------------  ----- 
 0   NIT                                                            15955 non-null  int64 
 1   FECHA CORTE                                                    15955 non-null  object
 2   Razón social de la sociedad                                    15955 non-null  object
 3   Corte de cuentas según estatutos                               15955 non-null  object
 4   Estado actual                                                  15955 non-null  object
 5   Clasificación Industrial Internacional Uniforme Versión 4 A.C  15955 non-null  object
 6   Tipo societario                                                15955 non-null  object
 7   La sociedad es                                                 1595

In [4]:
efe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16027 entries, 0 to 16026
Data columns (total 57 columns):
 #   Column                                                                                                                         Non-Null Count  Dtype  
---  ------                                                                                                                         --------------  -----  
 0   NIT                                                                                                                            16027 non-null  int64  
 1   FECHA CORTE                                                                                                                    16027 non-null  object 
 2   DURACION                                                                                                                       16027 non-null  object 
 3   (+/-) Ganancia (pérdida)                                                                                  

In [5]:
eri.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16005 entries, 0 to 16004
Data columns (total 19 columns):
 #   Column                                                       Non-Null Count  Dtype  
---  ------                                                       --------------  -----  
 0   Nit                                                          16005 non-null  int64  
 1   Fecha Corte                                                  16005 non-null  object 
 2   Duracion                                                     16005 non-null  object 
 3   Ingresos de actividades ordinarias                           16005 non-null  float64
 4   Costo de ventas                                              13633 non-null  float64
 5   Ganancia bruta                                               16005 non-null  float64
 6   Otros ingresos                                               16005 non-null  float64
 7   Gastos de ventas                                             11296 non-null 

In [6]:
esf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15988 entries, 0 to 15987
Data columns (total 64 columns):
 #   Column                                                                                                                                                                                                                  Non-Null Count  Dtype  
---  ------                                                                                                                                                                                                                  --------------  -----  
 0   Nit                                                                                                                                                                                                                     15988 non-null  int64  
 1   Fecha Corte                                                                                                                                            

In [7]:
ori.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16009 entries, 0 to 16008
Data columns (total 12 columns):
 #   Column                                                                                                           Non-Null Count  Dtype  
---  ------                                                                                                           --------------  -----  
 0   Nit                                                                                                              16009 non-null  int64  
 1   Fecha Corte                                                                                                      16009 non-null  object 
 2   Duracion                                                                                                         16009 non-null  object 
 3   Ganancia (pérdida)                                                                                               16005 non-null  float64
 4   Otro resultado integral, neto de i

A continuación se unen los archivos utilizando como llave el numero de identificación tributaria (NIT) de las empresas. Aquí, se debe notar que la variable identificadora se llama NIT para los elementos Caratula y efe y Nit para los elementos esf y ori, con lo cual se debe estandarizar el nombre.

Una de las buenas practicas cuando se trabaja con bases de datos relacionales es minimizar todas las columnas y reemplazar los espacios por un guión bajo ( _ ), esto facilitá luego autocompletar cuando se usa un IDE. 

In [8]:
def df_bajas_guion_bajo(df):
    '''
    Convierte las columnas de un DataFrame a bajas y transforma
    los espacios en guiones bajo 

    Args:
        df (pd.DataFrame) : El dataframe al que se le aplicará la transformación

    Returns:
        df (pd.DataFrame): El dataframe transformado

    '''   
    df.columns = df.columns.str.lower().str.replace(' ','_')
    return df

In [9]:
# Mapea función a todos los dataframes
caratula, efe, eri, esf, ori = map(df_bajas_guion_bajo, [caratula, efe, eri, esf, ori]) 

In [89]:
efe

,nit,fecha_corte,duracion,(+/-)_ganancia_(pérdida),(+/-)_ajustes_por_gastos_por_impuestos_a_las_ganancias,(_+_)_ajustes_por_gastos_de_depreciación_y_amortización,(+/-)_ajustes_por_deterioro_de_valor_(reversiones_de_pérdidas_por_deterioro_de_valor)_reconocidas_en_el_resultado_del_periodo,(+)_ajustes_por_provisiones,(+)_ajustes_por_costos_financieros,(+/-)_ajustes_por_pérdidas_(ganancias)_de_moneda_extranjera_no_realizadas,(+/-)_ajustes_por_pérdidas_(ganancias)_del_valor_razonable,(-)_ajustes_por_ganancias_no_distribuidas_de_asociadas,(+/-)_ajustes_por_pérdidas_(ganancias)_por_la_disposición_de_activos_no_corrientes,(+/-)_otros_ajustes_para_conciliar_la_ganancia_(pérdida),total_ajustes_para_conciliar_la_ganancia_(pérdida),(+/-)_ajustes_por_disminuciones_(incrementos)_en_los_inventarios,(+/-)_ajustes_por_la_disminución_(incremento)_de_cuentas_por_cobrar_de_origen_comercial,(+/-)_ajustes_por_disminuciones_(incrementos)_en_otras_cuentas_por_cobrar_derivadas_de_las_actividades_de_operación,(+/-)_ajustes_por_el_incremento_(disminución)_de_cuentas_por_pagar_de_origen_comercial,(+/-)_ajustes_por_incrementos_(disminuciones)_en_otras_cuentas_por_pagar_derivadas_de_las_actividades_de_operación,(+/-)_otras_entradas_(salidas)_de_efectivo,flujos_de_efectivo_netos_procedentes_de_(utilizados_en)_actividades_de_operación,(+)_flujos_de_efectivo_procedentes_de_la_pérdida_de_control_de_subsidiarias_u_otros_negocios,(-)_flujos_de_efectivo_utilizados_para_obtener_el_control_de_subsidiarias_u_otros_negocios,(+)_otros_cobros_por_la_venta_de_patrimonio_o_instrumentos_de_deuda_de_otras_entidades,(-)_otros_pagos_para_adquirir_patrimonio_o_instrumentos_de_deuda_de_otras_entidades,(+)_otros_cobros_por_la_venta_de_participaciones_en_negocios_conjuntos,(-)_otros_pagos_para_adquirir_participaciones_en_negocios_conjuntos,"(+)_importes_procedentes_de_la_venta_de_propiedades,_planta_y_equipo","(-)_compras_de_propiedades,_planta_y_equipo",(+)_importes_procedentes_de_ventas_de_activos_intangibles,(-)_compras_de_activos_intangibles,(+)_recursos_por_ventas_de_otros_activos_a_largo_plazo,(-)_compras_de_otros_activos_a_largo_plazo,"(-)_pagos_derivados_de_contratos_de_futuro,_a_término,_de_opciones_y_de_permuta_financiera","(+)_cobros_procedentes_de_contratos_de_futuro,_a_término,_de_opciones_y_de_permuta_financiera",(+)_dividendos_recibidos,(+)_intereses_recibidos,(+/-)_otras_entradas_(salidas)_de_efectivo.1,flujos_de_efectivo_netos_procedentes_de_(utilizados_en)_actividades_de_inversión,(+)_importes_procedentes_de_aumento_de_capital_y/o_recolocación_de_acciones,(-)_disminución_de_capital_social_y/o_readquisición_de_acciones,(-)_pagos_por_otras_participaciones_en_el_patrimonio,(+)_importe_procedente_del_aumento_prima_por_emisión,(-)_disminución_de_prima_por_emisión,(+)_importes_procedentes_de_préstamos,(-)_reembolsos_de_préstamos,(-)_pagos_de_pasivos_por_arrendamientos_financieros,(-)_dividendos_pagados,(-)_intereses_pagados,(+/-)_otras_entradas_(salidas)_de_efectivo.2,flujos_de_efectivo_netos_procedentes_de_(utilizados_en)_actividades_de_financiación,"incremento_(disminución)_neto_de_efectivo_y_equivalentes_al_efectivo,_antes_del_efecto_de_los_cambios_en_la_tasa_de_cambio",(+/-)_efectos_de_la_variación_en_la_tasa_de_cambio_sobre_el_efectivo_y_equivalentes_al_efectivo,incremento_(disminución)_neto_de_efectivo_y_equivalentes_al_efectivo,efectivo_y_equivalentes_al_efectivo_al_principio_del_periodo,efectivo_y_equivalentes_al_efectivo_al_final_del_periodo
0,800000090,31/12/2016,2016,12469.0,NaN,167834.0,NaN,NaN,NaN,NaN,NaN,NaN,66795.0,NaN,234629.0,-17838.0,-182132.0,NaN,-42904.0,228693.0,38158.0,271075.0,NaN,NaN,4500.0,NaN,NaN,NaN,NaN,133033.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-150461.0,-278994.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-7919.0,NaN,-7919.0,23134.0,15215.0
1,800000268,31/12/2016,2016,15955.0,-12380.0,126463.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,114083.0,4076.0,0.0,-3341.0,73974.0,-228532.0,NaN,-23785.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [123]:
# Unir todos los DataFrames
df_financial = reduce(lambda  left,right: pd.merge(left,right,on=['nit','fecha_corte', 'duracion'], how='inner', suffixes=('', '_y')), [efe, eri, ori])
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['nit','fecha_corte'], how='inner'), [caratula, esf, df_financial])
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16060 entries, 0 to 16059
Columns: 154 entries, nit to total_otro_resultado_integral_que_se_reclasificará_al_resultado_del_periodo,_neto_de_impuestos
dtypes: float64(132), int64(9), object(13)
memory usage: 19.0+ MB


A continuación se seleccionan las características sobre las cuales se aplicarán los algoritmos de clasificación y se analiza el tipo de cada variables mediante el comando `dtype`

In [120]:
columnas = ['nit', 'razón_social_de_la_sociedad', 'estado_actual',
 'clasificación_industrial_internacional_uniforme_versión_4_a.c',
 'tipo_societario', 'departamento_de_la_dirección_de_notificación_judicial',
 'ciudad_de_la_dirección_de_notificación_judicial','activos_corrientes_totales',
 'total_de_activos_no_corrientes','total_de_activos','pasivos_corrientes_totales',
 'total_de_pasivos_no_corrientes','total_pasivos','patrimonio_total',
 'ingresos_de_actividades_ordinarias','costo_de_ventas','ganancia_bruta',
 'otros_ingresos', 'gastos_de_ventas', 'gastos_de_administración', 'otros_gastos',
 'ganancia_(pérdida)_por_actividades_de_operación','ingresos_financieros',
 'costos_financieros','ganancia_(pérdida),_antes_de_impuestos','ganancia_(pérdida)']

new_col_names = ['nit','nombre','estado','ciiu','tipo_soc','dpto','ciudad',
                  'activo_cte','activo_no_cte','activo','pasivo_cte','pasivo_no_cte',
                  'pasivo','patrimonio','ingreso','costo_venta','ganancia_bruta',
                  'otro_ingreso','gasto_venta','gasto_admon','gasto_otro','ganancia_operacion',
                  'ingreso_fin','costo_fin','ganancia_antes_imp', 'ganancia']

In [126]:
datos = df_merged[columnas].copy()
datos.columns = new_col_names
datos.dtypes

nit                     int64
nombre                 object
estado                 object
ciiu                   object
tipo_soc               object
dpto                   object
ciudad                 object
activo_cte            float64
activo_no_cte         float64
activo                  int64
pasivo_cte            float64
pasivo_no_cte         float64
pasivo                  int64
patrimonio              int64
ingreso               float64
costo_venta           float64
ganancia_bruta        float64
otro_ingreso          float64
gasto_venta           float64
gasto_admon           float64
gasto_otro            float64
ganancia_operacion      int64
ingreso_fin           float64
costo_fin             float64
ganancia_antes_imp      int64
ganancia                int64
dtype: object

Se construyen algunos indicadores financieros adicionales y se calcula la cantidad de observaciones perdidas en el elemento datos:

In [128]:
datos['ingreso_activo'] = datos['ingreso']/datos['activo']
datos['ingreso_patrimonio'] = datos['ingreso']/datos['patrimonio']